## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

In [1]:
# Import the dataset from terminal
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray ./Resources/establishments.json

2023-05-25T23:03:58.178-0400	connected to: mongodb://localhost/
2023-05-25T23:03:58.181-0400	dropping: uk_food.establishments
2023-05-25T23:04:01.179-0400	[###########.............] uk_food.establishments	18.2MB/39.3MB (46.2%)
2023-05-25T23:04:03.567-0400	[########################] uk_food.establishments	39.3MB/39.3MB (100.0%)
2023-05-25T23:04:03.567-0400	39779 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'met', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': '27 Granville Road',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Lenox House',
 'BusinessType': 'Hotel/bed & breakfast/guest house',
 'BusinessTypeID': 7842,
 'ChangesByServerID': 0,
 'Distance': 4587.440906956158,
 'FHRSID': 632212,
 'LocalAuthorityBusinessID': 'PI/000043474',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DS',
 'RatingDate': '2020-03-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6470219ebb887170403af7fc'),
 'geocode': {'latitude': '51.152411', 'longitude': '1.385585'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/632212',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDat

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant ("Penang Flavours") that just opened in Greenwich, but hasn't been rated yet. Below, I included it in my analysis.

In [9]:
# Create a dictionary for the new restaurant data
new_halal_restaurant = {
                        "BusinessName":"Penang Flavours",
                        "BusinessType":"Restaurant/Cafe/Canteen",
                        "BusinessTypeID":"",
                        "AddressLine1":"Penang Flavours",
                        "AddressLine2":"146A Plumstead Rd",
                        "AddressLine3":"London",
                        "AddressLine4":"",
                        "PostCode":"SE18 7DY",
                        "Phone":"",
                        "LocalAuthorityCode":"511",
                        "LocalAuthorityName":"Greenwich",
                        "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
                        "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
                        "scores":{
                            "Hygiene":"",
                            "Structural":"",
                            "ConfidenceInManagement":""
                        },
                        "SchemeType":"FHRS",
                        "geocode":{
                            "longitude":"0.08384000",
                            "latitude":"51.49014200"
                        },
                        "RightToReply":"",
                        "Distance":4623.9723280747176,
                        "NewRatingPending":True
                        }

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_halal_restaurant)

In [11]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
establishments.find_one(query)

{'_id': ObjectId('647021a48fcbf199dcd89742'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1, 'BusinessType':1 }
results = establishments.find(query, fields)
pprint (list(results)[0:5])

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6470219ebb887170403af7fd')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6470219ebb887170403af7fe')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6470219ebb887170403af7ff')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6470219ebb887170403af800')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6470219ebb887170403af801')}]


3. Update the new restaurant with the `BusinessTypeID` found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"}, {'$set':{'BusinessTypeID':1}})

In [14]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
establishments.find_one(query)

{'_id': ObjectId('647021a48fcbf199dcd89742'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. Assuming that the magazine was not interested in any establishments in Dover: so I checked how many documents contained the Dover Local Authority. Then, removed any establishments within the Dover Local Authority from the database, and checked the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
Dover_LocalAuthorityName_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f"Dover_LocalAuthorityName_count: {Dover_LocalAuthorityName_count}")

Dover_LocalAuthorityName_count: 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName':'Dover'})

In [17]:
# Check if any remaining documents include Dover
Dover_LocalAuthorityName_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f"Dover_LocalAuthorityName_count: {Dover_LocalAuthorityName_count}")

Dover_LocalAuthorityName_count: 0


In [18]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6470219ebb887170403afae1'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. I observed that some of number values were stored as strings, when they should in fact be stored as numbers.

In [19]:
# Change the data type from String to Decimal for longitude 
establishments.update_many({},[{'$set':{'geocode.longitude':{'$toDouble':'$geocode.longitude'}}}])

In [20]:
# Change the data type from String to Decimal for latitude 
establishments.update_many({},[{'$set':{'geocode.latitude':{'$toDouble':'$geocode.latitude'}}}])

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [22]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},[{'$set':{'RatingValue':{'$toInt':'$RatingValue'}}}])

In [23]:
# Check that the coordinates and rating value are now numbers
query = {}
fields ={'geocode.longitude':1, 'geocode.latitude':1}
list(establishments.find(query,fields))[0:5]

[{'_id': ObjectId('6470219ebb887170403afae1'),
  'geocode': {'longitude': 1.196408, 'latitude': 51.086058}},
 {'_id': ObjectId('6470219ebb887170403afae2'),
  'geocode': {'longitude': 1.195625, 'latitude': 51.083812}},
 {'_id': ObjectId('6470219ebb887170403afae7'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('6470219ebb887170403afae8'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('6470219ebb887170403afae9'),
  'geocode': {'longitude': 1.194762, 'latitude': 51.085797}}]